In [1]:
import sqlite3
from datetime import datetime
from itertools import groupby

In [117]:
# Подключение к базе данных (файл будет создан, если не существует)
conn = sqlite3.connect('db/trading_bot.db')

# Создание курсора
cursor = conn.cursor()

# Создание таблицы сделок
cursor.execute('''
CREATE TABLE IF NOT EXISTS deals (
    id INTEGER PRIMARY KEY,
    algorithm_name TEXT NOT NULL,
    type INTEGER NOT NULL,
    instrument TEXT NOT NULL,
    datetime DATETIME DEFAULT CURRENT_TIMESTAMP,
    price REAL NOT NULL,
    commission REAL NOT NULL,
    total REAL NOT NULL
)
''')

# Создание индексов
cursor.execute('CREATE INDEX IF NOT EXISTS idx_instrument_datetime ON deals (algorithm_name, datetime)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_datetime ON deals (datetime)')

# Закрытие соединения
conn.close()

In [140]:
def get_daily_totals():
    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()
    
    # Получаем аггрегированные данные по дням и algorithm_name
    cursor.execute('''
    SELECT 
        algorithm_name, 
        strftime('%Y-%m-%d', SUBSTR(datetime, 1, 19)) AS day,
        SUM(total) AS daily_total,
        count(*) AS cnt
    FROM deals
    GROUP BY algorithm_name, day
    ORDER BY algorithm_name, day
    ''')
    
    aggregated_results = cursor.fetchall()
    
    # Итерация по аггрегированным результатам и выполнение дополнительного запроса для каждой пары algorithm_name, day
    for algorithm_name, day, daily_total, cnt in aggregated_results:
        cursor.execute('''
        SELECT type, total FROM deals
        WHERE algorithm_name = ? AND date(SUBSTR(datetime, 1, 19)) = date(?)
        ORDER BY datetime DESC
        LIMIT 1
        ''', (algorithm_name, day))
        
        last_operation_type, last_total = cursor.fetchone()
        
        # Если тип последней операции равен 1, вычитаем её total из итогового результата
        if last_operation_type == 1:           
            adjusted_total = daily_total - last_total
            print(f"День: {day}, {algorithm_name}, "
                  f"Подытог: {round(adjusted_total, 2)}, "
                  f"Записей: {cnt}")
        else:
            print(f"День: {day}, {algorithm_name}, "
                  f"Итог: {round(daily_total, 2)}, "
                  f"Записей: {cnt}")
    
    conn.close()

In [54]:
def add_deal(algorithm_name, type, instrument, price, commission, total):
    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()
    cursor.execute('''
    INSERT INTO deals (algorithm_name, type, instrument, price, commission, total)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (algorithm_name, type, instrument, price, commission, total))
    conn.commit()
    conn.close()

In [3]:
# добавление записи
# add_deal(f"RNFT_v2_c4_s2.py", 2, 'RNFT', 162.6, 0.9, 162.6-0.9)

In [143]:
def get_hourly_totals(date=None):
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')  # Текущая дата в формате строки

    conn = sqlite3.connect('db/trading_bot.db')
    cursor = conn.cursor()

    # Получаем данные за указанный день
    cursor.execute('''
    SELECT 
        algorithm_name,
        strftime('%H', SUBSTR(datetime, 1, 19)) AS hour,
        total,
        type
    FROM deals
    WHERE date(SUBSTR(datetime, 1, 19)) = ?
    ORDER BY algorithm_name, hour, datetime, id
    ''', (date,))

    results = cursor.fetchall()

    # группировка по имени алгоритма
    for algorithm_name, algorithm_group in groupby(results, key=lambda x: x[0]):
        algorithm_list = list(algorithm_group)
        
        print(f"Algorithm: {algorithm_name}")
        
        sum_price = 0
        prev_show = None
        # группировка часам
        for hour, hour_group in groupby(algorithm_list, key=lambda x: x[1]):
            hour_list = list(hour_group)
            
            hour_sum_price = 0
            operation_type = 0
            operation_total = 0 
            for _, _, operation_total, operation_type in hour_list:
                hour_sum_price += operation_total
            
            diff = hour_sum_price
            sum_price += hour_sum_price
            
            show_sum = sum_price
            # откидываем последнюю операцию, но только для локального вывода
            if operation_type == 1:
                show_sum -= operation_total
                # diff -= operation_total
                
            print(f"  {int(hour) + 3}: {round(show_sum, 2)} "
                  f"({round(show_sum - prev_show, 2) if prev_show is not None else '0'})")

            prev_show = show_sum                
            
    conn.close()

In [141]:
get_daily_totals()

День: 2024-03-14, v2_c4_s4, Подытог: 0.44, Записей: 5
День: 2024-03-14, v2_c4_s5, Итог: 0.74, Записей: 4
День: 2024-03-14, v2_c5_s4, Подытог: 0.54, Записей: 5
День: 2024-03-14, v2_c5_s5, Итог: 0.74, Записей: 4
День: 2024-03-14, v3_c4_s4, Итог: -0.04, Записей: 6
День: 2024-03-14, v3_c4_s5, Подытог: 0.44, Записей: 5
День: 2024-03-14, v3_c5_s4, Подытог: -0.14, Записей: 7
День: 2024-03-14, v3_c5_s5, Подытог: 0.36, Записей: 7


In [144]:
get_hourly_totals()  # Для текущего дня
# get_hourly_totals('2023-01-01')  # Для конкретной даты

Algorithm: v2_c4_s4
  14: 0.0 (0)
  15: 0.44 (0.44)
Algorithm: v2_c4_s5
  14: 0.0 (0)
  15: 0.74 (0.74)
Algorithm: v2_c5_s4
  14: 0.0 (0)
  15: 0.54 (0.54)
Algorithm: v2_c5_s5
  14: 0.0 (0)
  15: 0.74 (0.74)
Algorithm: v3_c4_s4
  14: 0.92 (0)
  15: -0.04 (-0.96)
Algorithm: v3_c4_s5
  14: 0.92 (0)
  15: 0.44 (-0.48)
Algorithm: v3_c5_s4
  14: 0.34 (0)
  15: -0.14 (-0.48)
Algorithm: v3_c5_s5
  14: 1.04 (0)
  15: 0.36 (-0.68)
